In [12]:
import django
import os

from django.core.wsgi import get_wsgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'django_orm_tests.settings')

application = get_wsgi_application()


import pandas as pd
from commerce.models import WithChoices
from django.db.models.functions import *
from django.db.models.expressions import *
from django.db.models.fields import *
from django.db.models import *
from commerce.models import *

In [60]:
from sqlalchemy import func, desc, asc, text
from sqlalchemy.sql import label, literal_column, literal

In [20]:
Cotizacion.sa.query(func.count(Cotizacion.sa.id), User.sa.username).join(Cotizacion.sa.creado).group_by(User.sa.id).all()

[(1, 'lautaro'),
 (1, 'usuario 0'),
 (1, 'usuario 1'),
 (2, 'usuario 3'),
 (1, 'usuario 4')]

In [22]:
Cotizacion.objects.values('creado__username').annotate(Count('id'))

<CotizacionQueryset [{'creado__username': 'lautaro', 'id__count': 1}, {'creado__username': 'usuario 0', 'id__count': 1}, {'creado__username': 'usuario 1', 'id__count': 1}, {'creado__username': 'usuario 3', 'id__count': 2}, {'creado__username': 'usuario 4', 'id__count': 1}]>

In [26]:
for value in (
    Cotizacion.objects
    .values('productos__nombre')
    .annotate(
        cantidad=Count('id'),
        total=Sum(F('productos__precio') * F('productos__cantidad')),
    )
): print(value)

{'productos__nombre': 'Producto 0', 'cantidad': 5, 'total': 304.4553197459108}
{'productos__nombre': 'Producto 1', 'cantidad': 4, 'total': 783.017031518781}
{'productos__nombre': 'Producto 10', 'cantidad': 6, 'total': 990.2370844671439}
{'productos__nombre': 'Producto 11', 'cantidad': 3, 'total': 901.525116563339}
{'productos__nombre': 'Producto 12', 'cantidad': 2, 'total': 92.69400655807588}
{'productos__nombre': 'Producto 13', 'cantidad': 5, 'total': 346.9551907843867}
{'productos__nombre': 'Producto 2', 'cantidad': 2, 'total': 552.0207134106386}
{'productos__nombre': 'Producto 3', 'cantidad': 4, 'total': 334.83972620114974}
{'productos__nombre': 'Producto 4', 'cantidad': 3, 'total': 26.76140386197111}
{'productos__nombre': 'Producto 5', 'cantidad': 2, 'total': 508.9299125825958}
{'productos__nombre': 'Producto 6', 'cantidad': 5, 'total': 247.1247165368071}
{'productos__nombre': 'Producto 7', 'cantidad': 2, 'total': 188.11342774200506}
{'productos__nombre': 'Producto 8', 'cantidad': 

In [50]:
(
    Cotizacion.sa
    .query(
        ProductoCotizado.sa.nombre,
        func.count(ProductoCotizado.sa.id).label('cantidad'),
        func.sum(ProductoCotizado.sa.precio * ProductoCotizado.sa.cantidad).label('total'),
    )
    .join(ProductoCotizado.sa.cotizacion)
    .group_by(ProductoCotizado.sa.nombre)
    .all()
)

[('Producto 0', 5, 304.4553197459108),
 ('Producto 1', 4, 783.017031518781),
 ('Producto 10', 6, 990.237084467144),
 ('Producto 11', 3, 901.525116563339),
 ('Producto 12', 2, 92.69400655807588),
 ('Producto 13', 5, 346.9551907843867),
 ('Producto 2', 2, 552.0207134106386),
 ('Producto 3', 4, 334.83972620114974),
 ('Producto 4', 3, 26.76140386197111),
 ('Producto 5', 2, 508.9299125825958),
 ('Producto 6', 5, 247.1247165368071),
 ('Producto 7', 2, 188.11342774200506),
 ('Producto 8', 5, 260.90219605975767),
 ('Producto 9', 2, 110.38411431409338)]

-----------------------------------------

----------------

In [40]:
for value in (
    Cotizacion.objects
    .values('productos__nombre')
    .annotate(
        cantidad=Count('id'),
        total=Sum(F('productos__precio') * F('productos__cantidad')),
        ranking=Window(expression=DenseRank(), order_by=[F('total').desc()]),
    )
    .order_by('ranking')
): print(value)

{'productos__nombre': 'Producto 10', 'cantidad': 6, 'total': 990.2370844671439, 'ranking': 1}
{'productos__nombre': 'Producto 11', 'cantidad': 3, 'total': 901.525116563339, 'ranking': 2}
{'productos__nombre': 'Producto 1', 'cantidad': 4, 'total': 783.017031518781, 'ranking': 3}
{'productos__nombre': 'Producto 2', 'cantidad': 2, 'total': 552.0207134106386, 'ranking': 4}
{'productos__nombre': 'Producto 5', 'cantidad': 2, 'total': 508.9299125825958, 'ranking': 5}
{'productos__nombre': 'Producto 13', 'cantidad': 5, 'total': 346.9551907843867, 'ranking': 6}
{'productos__nombre': 'Producto 3', 'cantidad': 4, 'total': 334.83972620114974, 'ranking': 7}
{'productos__nombre': 'Producto 0', 'cantidad': 5, 'total': 304.4553197459108, 'ranking': 8}
{'productos__nombre': 'Producto 8', 'cantidad': 5, 'total': 260.90219605975767, 'ranking': 9}
{'productos__nombre': 'Producto 6', 'cantidad': 5, 'total': 247.1247165368071, 'ranking': 10}
{'productos__nombre': 'Producto 7', 'cantidad': 2, 'total': 188.11

In [73]:
(
    Cotizacion.sa
    .query(
        ProductoCotizado.sa.nombre,
        func.count(ProductoCotizado.sa.id).label('cantidad'),
        func.sum(ProductoCotizado.sa.precio * ProductoCotizado.sa.cantidad).label('total'),
        func.dense_rank().over(order_by=func.sum(ProductoCotizado.sa.precio * ProductoCotizado.sa.cantidad).desc())
    )
    .join(ProductoCotizado.sa.cotizacion)
    .group_by(ProductoCotizado.sa.nombre)
    .all()
)

[('Producto 10', 6, 990.237084467144, 1),
 ('Producto 11', 3, 901.525116563339, 2),
 ('Producto 1', 4, 783.017031518781, 3),
 ('Producto 2', 2, 552.0207134106386, 4),
 ('Producto 5', 2, 508.9299125825958, 5),
 ('Producto 13', 5, 346.9551907843867, 6),
 ('Producto 3', 4, 334.83972620114974, 7),
 ('Producto 0', 5, 304.4553197459108, 8),
 ('Producto 8', 5, 260.90219605975767, 9),
 ('Producto 6', 5, 247.1247165368071, 10),
 ('Producto 7', 2, 188.11342774200506, 11),
 ('Producto 9', 2, 110.38411431409338, 12),
 ('Producto 12', 2, 92.69400655807588, 13),
 ('Producto 4', 3, 26.76140386197111, 14)]